<a href="https://colab.research.google.com/github/aarshitaacharya/peft-techniques/blob/main/R13_Prompt_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install transformers peft datasets dash pyngrok

# Import required libraries
import os
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import PromptTuningConfig, get_peft_model, TaskType
from datasets import load_dataset
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from pyngrok import ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Load dataset
dataset = load_dataset("THUDM/humaneval-x", "js")

# Model and tokenizer configuration
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side='right'

# Load foundation model
foundation_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

humaneval-x.py:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

js/test/0000.parquet:   0%|          | 0.00/106k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
# Tokenization function
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['prompt'], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()
    return tokenized_inputs

# Tokenize the dataset
tokenized_dataset = dataset['test'].map(tokenize_function, batched=True)

# Split the dataset into training (80%) and evaluation (20%) sets
train_size = int(0.8 * len(tokenized_dataset))
train_indices = random.sample(range(len(tokenized_dataset)), train_size)
train_dataset = tokenized_dataset.select(train_indices)
eval_indices = list(set(range(len(tokenized_dataset))) - set(train_indices))
eval_dataset = tokenized_dataset.select(eval_indices)

Map:   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
# Prompt Tuning configuration
prompt_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    num_virtual_tokens=8,
    tokenizer_name_or_path=model_name
)

# Apply Prompt Tuning
prompt_model = get_peft_model(foundation_model, prompt_config)

# Training arguments
output_directory = "./prompt_tuning_outputs"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

training_args = TrainingArguments(
    output_dir=output_directory,
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    save_steps=500,
    save_total_limit=2,
)

# Trainer setup
trainer = Trainer(
    model=prompt_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# Start fine-tuning
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


TrainOutput(global_step=40, training_loss=2.995456886291504, metrics={'train_runtime': 120.6195, 'train_samples_per_second': 5.43, 'train_steps_per_second': 0.332, 'total_flos': 590657643085824.0, 'train_loss': 2.995456886291504, 'epoch': 4.848484848484849})

In [ ]:
# Define the app for model inference
app = Dash(__name__)

app.layout = html.Div(style={'backgroundColor': '#f8f9fa', 'padding': '20px'}, children=[
    html.H1("JavaScript Code Generation Model Inference", style={'textAlign': 'center', 'color': '#343a40'}),
    dcc.Input(id='input-text', type='text', placeholder='Enter prompt for JavaScript code generation...',
               style={'width': '100%', 'padding': '10px', 'fontSize': '18px', 'marginBottom': '10px'}),
    html.Button('Submit', id='submit-button', n_clicks=0,
                 style={'backgroundColor': '#007bff', 'color': 'white', 'padding': '10px 20px',
                        'border': 'none', 'borderRadius': '5px', 'cursor': 'pointer'}),
    html.Div(id='output-prediction', style={'marginTop': '20px', 'fontSize': '18px',
                                             'color': '#495057', 'border': '1px solid #ced4da',
                                             'padding': '10px', 'borderRadius': '5px'})
])

# Define callback for model inference
@app.callback(
    Output('output-prediction', 'children'),
    [Input('submit-button', 'n_clicks')],
    [Input('input-text', 'value')]
)
def update_output(n_clicks, input_text):
    if n_clicks > 0 and input_text:
        prompt = f"// Generate a JavaScript function for the following task:\n{input_text}\n"
        inputs = tokenizer(prompt, return_tensors='pt').to(prompt_model.device)

        with torch.no_grad():
            outputs = prompt_model.generate(**inputs, max_length=150)

        decoded_prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return html.Pre(f"Generated Code:\n{decoded_prediction}", style={'whiteSpace': 'pre-wrap'})

    return "Enter a prompt and click Submit."

# Start the app server
app.run_server(port=8050)

<IPython.core.display.Javascript object>

In [ ]:
# Start ngrok tunnel for public access
ngrok.set_auth_token('')  # Insert your ngrok auth token
public_url = ngrok.connect(8050)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://1ff1-34-125-153-45.ngrok-free.app" -> "http://localhost:8050"
